## Generating Synthetic Test Datasets

**Why use Synthetic Test Datasets?**

Evaluating the performance of RAG (Retrieval-Augmented Generation) augmented pipelines is crucial.

However, manually creating hundreds of QA (Question-Answer-Context) samples from documents can be time-consuming and labor-intensive. Additionally, human-generated questions may struggle to reach the level of complexity needed for thorough evaluation, ultimately affecting the quality of the assessment.

Using synthetic data generation can reduce developer time in the data aggregation process **by up to 90%**.

    RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html


In [ ]:
import pandas as pd

df = pd.read_json("data/qa_dataset.jsonl", lines=True)
df.head()

## Document Used for Practice

Amazon Bedrock Manual Documentation (https://docs.aws.amazon.com/bedrock/latest/userguide/)

- Link: https://d1jp7kj5nqor8j.cloudfront.net/bedrock-manual.pdf
- File name: `bedrock-manual.pdf`

_Please copy the downloaded file to the data folder for the practice session_

In [ ]:
from datasets import Dataset

subset_length = 10
test_dataset = Dataset.from_pandas(df.head(subset_length))

## Document Preprocessing

In [ ]:
import ast
import re

def clean_string(s):
    s = re.sub(r'[^\x00-\x7F]+', '', s)
    s = s.replace("'", '"')
    return s

def convert_to_list(example):
    cleaned_context = clean_string(example["contexts"])
    try:
        contexts = ast.literal_eval(cleaned_context)
    except:
        contexts = cleaned_context
    return {"contexts": contexts}

test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

## Test Q&A Dataset Generation

In [ ]:
test_dataset[0]['question']

### Tool Use 

LLM will generate Q&A dataset that conforms to the schema description in the tooluse config.

In [ ]:
# RAG implementation sample 1 (Replace with RAG pipeline for evaluation)
from libs.bedrock_kb_util import context_retrieval_from_kb

question = test_dataset[0]['question']
search_result = context_retrieval_from_kb(question, 3, 'us-west-2', 'CNDSUOPKAS', 'SEMANTIC')
print(search_result[0])

contexts = "\n--\n".join([result['content'] for result in search_result])

## Q&A Dataset Generation Instruction

- `simple`: directly answerable questions from the given context
- `complex`: reasoning questions and answers.

_Modify the system/user prompts tailored to your dataset_

Generated Q&A pair will be stored in `data/qa_dataset.jsonl`

In [ ]:
print(contexts)

In [ ]:
import boto3
from botocore.config import Config

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
region = 'us-west-2'

retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", config=retry_config)

In [ ]:
def generate_answer(question, contexts):
    system_prompt = """You are an AI assistant that uses retrieved context to answer questions accurately. 
    Follow these guidelines:
    1. Use the provided context to inform your answers.
    2. If the context doesn't contain relevant information, say "I don't have enough information to answer that."
    3. Be concise and to the point in your responses."""

    user_prompt = f"""Context: {contexts}

    Question: {question}

    Please answer the question based on the given context."""

    response = boto3_client.converse(
        modelId=model_id,
        messages=[{'role': 'user', 'content': [{'text': user_prompt}]}],
        system=[{'text': system_prompt}]
    )

    answer = response['output']['message']['content'][0]['text']
    return answer

In [ ]:
test_dataset

In [ ]:
from tqdm import tqdm
from time import sleep

kb_region = 'us-west-2'
kb_id = 'CNDSUOPKAS'
top_k = 3

def process_item(item):
    sleep(5)  # Prevent throttling
    question = item['question']
    search_result = context_retrieval_from_kb(question, top_k, kb_region, kb_id, 'SEMANTIC')

    contexts = [result['content'] for result in search_result]
    answer = generate_answer(question, "\n--\n".join(contexts))

    return {
        'question': item['question'],
        'ground_truth': item['ground_truth'],
        'original_contexts': item['contexts'],
        'retrieved_contexts': contexts,
        'answer': answer
    }

updated_dataset = test_dataset.map(process_item)

In [ ]:
import json
output_file = "data/updated_qa_dataset.jsonl"

with open(output_file, 'w', encoding='utf-8') as f:
    for item in updated_dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"Dataset saved to {output_file}")

In [1]:
import json
from datasets import Dataset

input_file = "data/updated_qa_dataset.jsonl"
def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            yield json.loads(line.strip())

updated_dataset = Dataset.from_list(list(read_jsonl(input_file)))

item = updated_dataset[0]
print(f"Question: {item['question']}")
print(f"Answer: {item['answer']}")
print("-" * 50)

/home/ec2-user/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Question: How do temperature, Top K, and Top P parameters interact in Amazon Bedrock's foundation models, and how might adjusting these affect the output when generating text about different types of equines?
Answer: Based on the provided context, here's how temperature, Top K, and Top P parameters interact in Amazon Bedrock's foundation models, and how adjusting them might affect the output when generating text about different types of equines:

1. Temperature: 
- Lower values increase the likelihood of higher-probability tokens and decrease the likelihood of lower-probability tokens. This would make the model more likely to choose "horses" in the given example.
- Higher values increase the likelihood of lower-probability tokens and decrease the likelihood of higher-probability tokens. This would make the model more likely to consider "zebras" or even "unicorns" in the example.

2. Top K:
- A lower Top K value (e.g., 2) would limit the model to consider only the top K most likely cand

In [3]:
from libs.eval_metrics import (
    evaluate,
    AnswerRelevancy, 
    Faithfulness, 
    ContextRecall,
    ContextPrecision
)

llm_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
emb_id = "amazon.titan-embed-text-v2:0"
region = "us-west-2"

metrics = [AnswerRelevancy, Faithfulness, ContextRecall, ContextPrecision]

def map_dataset(example):
    return {
        "user_input": example["question"],
        "retrieved_contexts": example["retrieved_contexts"],
        "referenced_contexts": example["original_contexts"],
        "response": example["answer"],
        "reference": example["ground_truth"]
    }

dataset = updated_dataset.map(map_dataset).select(range(2))
results = evaluate(dataset, metrics, llm_id, emb_id, region)
print(results)

Map: 100%|██████████| 10/10 [00:00<00:00, 1941.63 examples/s]


[1, 1, 1, 1, 1, 1, 0]
[1]
[1, 1, 1, 1, 1]
[1]


In [8]:
import pandas as pd 

result_df = pd.DataFrame([results])
result_df.to_csv("data/ragas_evaluation_result.csv", index=False)
print(result_df)

   AnswerRelevancy  Faithfulness  ContextRecall  ContextPrecision
0         0.977534      0.928571            1.0          0.791667
